In [ ]:
#google driveとの連携
from google.colab import drive
drive.mount('/content/drive')

#カレントディレクトリの設定
dir_path = '/content/drive/My Drive/Colab Notebooks/repo10/'
import os
os.chdir(dir_path)

#モジュール追加用pathの設定
import sys
sys.path.append('..')

#モジュールのインポート
import numpy as np

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**80.py**

In [ ]:
import re
from collections import Counter
import pickle

# 単語の最小出現回数
min_freq = 2

train_path = './data/train.txt'#todo 学習用ファイルパス
word2id_path = './data/train.pt' #todo 出力先ファイルパス


# タイトルに含まれる単語をすべてリストword_listに格納
word_list = []
with open(train_path, encoding="utf-8") as f:
    for line in f:
        if not line:
            continue
        title = line.split('\t')[1]#todo タイトルを取得
        for word in title.split():
            word_list.append(word)

word_count = Counter(word_list)#todo Counterを使ってword_listの各単語の出現回数をカウント
# print('word_count', word_count) #word_count Counter({'to': 2098, '...': 1991, 'in': 1424, ...
# 単語をIDに変換するためのdict．keyを単語，valueをIDにする．
word2id = {}

id = 1
for word, freq in sorted(word_count.items(), key=lambda x:x[1], reverse=True):
    if freq < min_freq: #todo その単語の出現頻度がmin_freqよりも小さい場合:
        word2id[word] = 0#todo 単語のIDは0とする
    else:
        word2id[word] = id #todo 単語にIDを割り当てる
        id += 1#todo IDをインクリメント

        
# 表示
print("word_num", len(word2id))
# 保存
pickle.dump(word2id, open(word2id_path, "wb"))


word_num 23620


**83.py**

In [ ]:
print(word2id)

{'to': 1, '...': 2, 'in': 3, 'on': 4, 'as': 5, 'UPDATE': 6, '-': 7, 'for': 8, 'of': 9, 'The': 10, 'US': 11, 'To': 12, 'and': 13, 'the': 14, 'In': 15, 'Of': 16, 'at': 17, 'a': 18, 'With': 19, 'For': 20, 'Is': 21, 'A': 22, 'with': 23, 'after': 24, 'And': 25, 'New': 26, 'Kardashian': 27, 'On': 28, 'by': 29, 'After': 30, 'Kim': 31, 'says': 32, 'up': 33, 'At': 34, 'is': 35, 'From': 36, 'China': 37, 'from': 38, 'new': 39, '(1)': 40, 'Says': 41, 'her': 42, 'Miley': 43, 'Fed': 44, 'ECB': 45, 'shares': 46, 'Cyrus': 47, 'About': 48, 'First': 49, 'CEO': 50, 'West': 51, 'May': 52, 'over': 53, 'Over': 54, 'Will': 55, 'Chris': 56, 'Kanye': 57, 'St': 58, 'Be': 59, 'More': 60, 'You': 61, 'she': 62, 'As': 63, 'be': 64, 'bln': 65, 'Up': 66, 'data': 67, 'Justin': 68, "'The": 69, 'Ukraine': 70, 'Are': 71, 'off': 72, 'euro': 73, 'GLOBAL': 74, 'but': 75, 'Stocks': 76, 'Google': 77, 'Billion': 78, 'sales': 79, 'are': 80, 'profit': 81, '2014': 82, 'Bieber': 83, 'out': 84, 'Bank': 85, 'Not': 86, '&': 87, 'It':

In [ ]:
import pickle
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence, pad_sequence, pack_padded_sequence

In [ ]:
verbose = False

word2id_path = './data/train.pt' #todo 80.pyで保存したファイルパスを指定
word2id = pickle.load(open(word2id_path, "rb"))

# word_listの単語リストをIDリストに変換する関数
def get_ids(word_list):
    ids = []
    for word in word_list:
        if word in word2id: #todo wordがword2idのキーに含まれる場合，
            ids.append(word2id[word]) #todo そのIDをidsにappend
        else:
            ids.append(0) #todo wordがword2idのキーに含まれない場合，0をidsにappend
    return ids


def get_data(fname):
    label_list = [] # ラベル(カテゴリーのid)を格納するリスト
    ids_list = [] # タイトルに含まれる単語のID系列のtensorを格納するリスト

    category2num = {"b": 0, "t": 1, "e": 2, "m": 3}

    with open(fname) as f:
        for line in f:
            if not line:
                continue
            line = line.strip()
            title = line.split('\t')[1]#todo lineからタイトルを取得
            word_list =  title.split() #todo titleを単語に分割
            ids = get_ids(word_list)
            ids_tensor = torch.tensor(ids)#todo idsをtensorに変換
            ids_list.append(ids_tensor)#todo ids_tensorをids_listにappend

            category = line.split('\t')[0]#todo lineからカテゴリーを取得
            label = category2num[category] #todo category2numでカテゴリーをラベルに変換
            label_list.append(label) #todo labelをlabel_listにappend
          
    labels = torch.tensor(label_list) #todo label_listをtensorに変換
    return ids_list, labels


class ClassifierRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, target_size):
        super(ClassifierRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)#todo 単語埋め込み層
        self.rnn = nn.LSTM(embedding_dim, hidden_dim) #todo LSTM層
        self.linear = nn.Linear(hidden_dim, target_size) #todo 線形層
        self.softmax = nn.Softmax()
    
    def forward(self, x, x_len):  
        embed_x = self.embedding(x) #todo xを単語埋め込み

        packed_x = pack_padded_sequence(embed_x, x_len, batch_first=True, enforce_sorted=False) #todo embed_xとx_lenを用いてPackedSequence型のデータに変換

        _, (h, _) = self.rnn(packed_x) #todo LSTM層に入力
        output = self.linear(h) #todo 線形層に入力

        output = np.squeeze(output, 0) #todo squeeze()を使用し，outputの形状が[1, batch_size, target_size]となっているものを[batch_size, target_size]に変換

        #print(output.size()) #確認用(確認する場合はコメントアウトを解除)
        output = self.softmax(output)
        return output


class RnnDataset(torch.utils.data.Dataset):
    def __init__(self, data, label):
        self.length = [len(data[i]) for i in range(len(data))] #todo dataに含まれる各ID系列の長さのリスト
        self.label = label
        
        self.pad_data = pad_sequence(data, batch_first=True) #todo dataをpadding

    def __len__(self):
        return len(self.pad_data)

    def __getitem__(self, idx):
        return self.pad_data[idx], self.length[idx], self.label[idx] #todo idxに対応するpad_data, length, labelの3つを返り値とする



emb_dim = 300
hidden_dim = 50
target_size = 4
batch_size = 64

vocab_size = max(word2id.values()) + 1 #0のやつはカウントしない

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# データの準備
train_path = './data/train.txt'#todo 学習用ファイルパス
train_x, train_y = get_data(train_path)
valid_path = './data/valid.txt' #todo 評価用ファイルパス
valid_x, valid_y = get_data(valid_path)

###チェック用###
if verbose:
  print('len(train_x)', len(train_x)) #10671
  print('len(train_y)', len(train_y)) #10671
  print(train_x[0]) #tensor([1, 0, 2, 3, 4, 5, 6])
  print(train_x[1]) #tensor([ 7,  8,  0,  9, 10,  0,  0, 11, 12, 13])
  print(train_x[2]) #tensor([14, 15,  9, 16, 17,  0, 18, 19, 20, 21, 22])
  print(train_x[3]) #tensor([23, 24,  0,  8, 25, 26, 27, 28, 29, 30])
  print(train_y[0]) #tensor(3) m
  print(train_y[1]) #tensor(0) b
  print(train_y[2]) #tensor(1) t
  print(train_y[3]) #tensor(0) b
  print('-'*30)

dataset = RnnDataset(train_x, train_y)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

###チェック用###
if verbose:
  print('---dataset = RnnDataset(train_x, train_y)---')
  print(dataset.__getitem__(0)) #(tensor([1, 0, 2, 3, 4, 5, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 7, tensor(3))
  print(dataset.__getitem__(1)) #(tensor([ 7,  8,  0,  9, 10,  0,  0, 11, 12, 13,  0,  0,  0,  0,  0,  0,  0,  0, 0,  0]), 10, tensor(0))
  print(dataset.__getitem__(2)) #(tensor([14, 15,  9, 16, 17,  0, 18, 19, 20, 21, 22,  0,  0,  0,  0,  0,  0,  0, 0,  0]), 11, tensor(1))
  print(dataset.__getitem__(3)) #(tensor([23, 24,  0,  8, 25, 26, 27, 28, 29, 30,  0,  0,  0,  0,  0,  0,  0,  0, 0,  0]), 10, tensor(0))
  print(max(dataset.length)) #20

model = ClassifierRNN(vocab_size, emb_dim, hidden_dim, target_size)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=0.05)
loss_fn = nn.CrossEntropyLoss()


def train(model, train_loader, len_train):
    model.train()
    total_loss = 0
    correct_num = 0
    len_loader = len(train_loader)

    for data, data_len, target in train_loader:
        # GPUへ
        data = data.to(device)
        data_len = data_len.to(device)
        target = target.to(device)

        optimizer.zero_grad()
        pred = model.forward(data, data_len) #todo modelで予測
        #print(pred)
        loss = loss_fn(pred, target) #todo lossを計算
        loss.backward()
        optimizer.step()
        total_loss += loss #todo lossをtotal_lossに加算
        correct_num += (torch.argmax(pred, axis=1) == target).sum().item() #todo 予測の正解数をカウント
    
    batch_loss = total_loss/len_train #バッチごとの平均ロス
    acc = correct_num/len_train #todo 正解率
    return batch_loss, acc


# validデータはそのままGPUへ
valid_x_len = [len(valid_x[i]) for i in range(len(valid_x))]#todo valid_xの各要素の長さのリスト

###チェック用###
if verbose:
  print('len(valid_x)', len(valid_x)) #1334
  print('len(valid_y)', len(valid_y)) #13341
  print(valid_x[0]) #tensor([1513, 1514,  867,  306, 1516,   90, 4541,    0,    0, 4542])
  print(len(valid_x[0])) #10
  print('valid_x_len', valid_x_len) #[10, 12, 11, 12, 10, 10, 11, 11, 11, 10, 10, 8, 9, 10, 11, 10, 13, 10, 12, 10, 13, 12, 9, 8, 12...
valid_pad_x = pad_sequence(valid_x, batch_first=True)#todo valid_xをpadding

valid_x_len_tensor = torch.tensor(valid_x_len)

# GPUへ
valid_x_len = valid_x_len_tensor.to(device)
valid_pad_x = valid_pad_x.to(device)
valid_y = valid_y.to(device)

def evaluation(model, data, data_len, target):
    model.eval()
    with torch.no_grad():
        pred = model(data, data_len) #todo 予測
        loss = loss_fn(pred, target) #todo 損失を計算
        acc = (torch.argmax(pred, axis=1) == target).sum().item() / len(pred) #todo 正解率を計算
    return loss.item(), acc

len_train = len(train_y)

for epoch in range(30):
    train_loss, train_acc = train(model, train_loader, len_train)
    valid_loss, valid_acc = evaluation(model, valid_pad_x, valid_x_len, valid_y)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    print(f"<valid> Loss: {valid_loss}\tAccuracy: {valid_acc}")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:61: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 0
<train> Loss: 0.021015863865613937	Accuracy: 0.41654952675475587
<valid> Loss: 1.3041681051254272	Accuracy: 0.44377811094452774
epoch: 1
<train> Loss: 0.020129363983869553	Accuracy: 0.46490488239152844
<valid> Loss: 1.2608531713485718	Accuracy: 0.49325337331334335
epoch: 2
<train> Loss: 0.019582806155085564	Accuracy: 0.5253490769374941
<valid> Loss: 1.2324703931808472	Accuracy: 0.5359820089955023
epoch: 3
<train> Loss: 0.01918732188642025	Accuracy: 0.5513072814169244
<valid> Loss: 1.2121855020523071	Accuracy: 0.5442278860569715
epoch: 4
<train> Loss: 0.01889723539352417	Accuracy: 0.5573048449067566
<valid> Loss: 1.1975094079971313	Accuracy: 0.5472263868065967
epoch: 5
<train> Loss: 0.0186734888702631	Accuracy: 0.562271577171774
<valid> Loss: 1.1858558654785156	Accuracy: 0.5539730134932533
epoch: 6
<train> Loss: 0.018484361469745636	Accuracy: 0.571361634336051
<valid> Loss: 1.1750961542129517	Accuracy: 0.568215892053973
epoch: 7
<train> Loss: 0.018296049907803535	Accuracy: 0.58

わずかに過学習しているが問題ない範囲だと判断した。epoch数を増やせば精度が上がると考えられる。

In [ ]:
for epoch in range(100):
    train_loss, train_acc = train(model, train_loader, len_train)
    valid_loss, valid_acc = evaluation(model, valid_pad_x, valid_x_len, valid_y)

    print(f"epoch: {epoch}")
    print(f"<train> Loss: {train_loss}\tAccuracy: {train_acc}")
    print(f"<valid> Loss: {valid_loss}\tAccuracy: {valid_acc}")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 0
<train> Loss: 0.015063450671732426	Accuracy: 0.7877424796176553
<valid> Loss: 0.9819101691246033	Accuracy: 0.7631184407796102
epoch: 1
<train> Loss: 0.015018952079117298	Accuracy: 0.7896167182082279
<valid> Loss: 0.9802885055541992	Accuracy: 0.7638680659670165
epoch: 2
<train> Loss: 0.01497625932097435	Accuracy: 0.7920532283759723
<valid> Loss: 0.9779528379440308	Accuracy: 0.7653673163418291
epoch: 3
<train> Loss: 0.014942076057195663	Accuracy: 0.7933651953893731
<valid> Loss: 0.977223813533783	Accuracy: 0.7668665667166417
epoch: 4
<train> Loss: 0.014914860017597675	Accuracy: 0.7949582981913598
<valid> Loss: 0.9752698540687561	Accuracy: 0.7683658170914542
epoch: 5
<train> Loss: 0.014885990880429745	Accuracy: 0.7955205697685316
<valid> Loss: 0.9792079925537109	Accuracy: 0.7661169415292354
epoch: 6
<train> Loss: 0.01487733144313097	Accuracy: 0.795895417486646
<valid> Loss: 0.9740456938743591	Accuracy: 0.7698650674662668
epoch: 7
<train> Loss: 0.014845798723399639	Accuracy: 0.796

epoch数を上げると確かに精度が上がったが、その分validとの差が開き、過学習の度合いが上がった。

Reference
1. https://note.nkmk.me/python-collections-counter/
2. https://www.headboost.jp/python-counter/#2_Counter
3. https://note.nkmk.me/python-numpy-squeeze/